In [1]:
# This code was made with the youtube tutorial by Xanadu: How to solve the QUBO problem | PennyLane Tutorial

# Events organization

# 5 events: E1, E2, E3, E4, E5
# 4 people: P1, P2, P3, P4

# Preferences:

    # P1 want to go to  E1 and E4
    # P2 want to go to E2 and E3
    # P3 want to go to E4 and E5
    # P4 want to go to E3 and E4

# Binary variables
    
    # xi = 0 if the i-th event is on Saturday
    # xi = 1 if the i-th event is on Sunday

# Formulation

    # f(x1, x2, x3, x4, x5) = 4 + x1 + x2 + 2x3 +3x4 + x5 - 2x1x4 - 2x2x3 - 2x4x5 - 2x3x4
    
# Change of variable due to quantum computing

    # xi = (1 - zi)/2
    
    # zi = 1 if the event is on Saturday
    # zi = -1 if the event is on Sunday
    
# New formulation

    # f(z1,z2,z3,z4,z5) = 6 - 1/2(z1z4) - 1/2(z2z3) - 1/2(z4z5) - 1/2(z3z4)


In [3]:
import pennylane as qml
from pennylane import numpy as np

# H = Hamiltonian
# Change the signs to minimize the expected value, by that, we obtain the maximum of the other formulation
H = - 6 * qml.Identity(1) + \
    0.5 * qml.PauliZ(1) @ qml.PauliZ(4) + \
    0.5 * qml.PauliZ(2) @ qml.PauliZ(3) + \
    0.5 * qml.PauliZ(4) @ qml.PauliZ(5) + \
    0.5 *qml.PauliZ(3) @ qml.PauliZ(4)

print(H)

  (-6.0) [I1]
+ (0.5) [Z1 Z4]
+ (0.5) [Z2 Z3]
+ (0.5) [Z4 Z5]
+ (0.5) [Z3 Z4]


In [5]:
dev = qml.device("default.qubit", wires = H.wires)

@qml.qnode(dev)
def circuit(params):
    
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires = wire)
        
    return qml.expval(H)

circuit([0,0,0,0,0])

tensor(-4., requires_grad=True)

In [6]:
params = np.random.rand(len(H.wires))
opt  = qml.AdagradOptimizer(stepsize = 0.5)
epochs = 200

for epoch in range(epochs):
    params = opt.step(circuit, params)
    
circuit(params)

tensor(-8., requires_grad=True)

In [7]:
dev = qml.device("default.qubit", wires = H.wires, shots=1)

@qml.qnode(dev)
def circuit(params):
    
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires = wire)
        
    return qml.sample()

circuit(params)

# Response: [1,0,1,0,1]
# E1, E3, E5 happens on Sunday
# E2, E4 happens on Saturday

array([1, 0, 1, 0, 1], dtype=int64)